In [1]:
import os
import random
import pandas as pd
import yaml
yaml.Dumper.ignore_aliases = lambda *args : True
import itertools
pd.set_option('display.max_colwidth', None)

In [2]:
random.seed(1)

In [3]:
models =['Clean', 'Noisy', 'DCUNet', 'DPTNet', 'RegressionFCNN', 'SMoLnet', 'WaveUNet']
snrs = ['-10', '-15', '-20', '-25']
df_all_snr_dict = {
#     '0': '', 
#     '-5': '',
    '-10': '',
    '-15': '',
    '-20': '',
    '-25': '',
#     '-30' : ''
}

In [4]:
meta_data_dir = 'meta_data_for_yaml'
for snr in snrs:
    df_all_snr_dict[str(snr)] = pd.read_csv(f'meta_data_for_yaml/website_meta_data_snr_{snr}dB.csv', usecols = ['Clean', 'DCUNet', 'DPTNet',  'SMoLnet', 'WaveUNet'])

In [5]:
#get big dataframe
# df = pd.concat([df_all_snr_dict['0'], df_all_snr_dict['-5'], df_all_snr_dict['-10'], df_all_snr_dict['-15'], df_all_snr_dict['-20'], df_all_snr_dict['-25'], df_all_snr_dict['-30']], ignore_index=True)

df = pd.concat([df_all_snr_dict['-10'], df_all_snr_dict['-15'], df_all_snr_dict['-20'], df_all_snr_dict['-25']], ignore_index=True)

In [6]:
def get_combinations_of_index(df, idx):
    #all wavs except for the input to create combinations
    #Input will be our reference
    denoised_list = list(df.iloc[idx])[1:]
    #create pairs of twos
    #e.g. - (WaveUNet_SX139.WAV.n121.wav, RegressionFCNN_SX139.WAV.n121.wav) ...
    combinations = list(itertools.combinations(denoised_list, 2))

    # Final combination list with input appended to all 
    #e.g. - (input.wav, WaveUNet_SX139.WAV.n121.wav, RegressionFCNN_SX139.WAV.n121.wav) ...
    final_combination_list = []
    for c in combinations:
        #trick to append to tuple 
        c = (df.iloc[idx][0], *c)
        final_combination_list.append(c)
    return final_combination_list    
    

In [7]:
final_list = []

In [8]:
for i in range(len(df)):
    res = get_combinations_of_index(df, i)
    for r in res:
        final_list.append(r)

In [9]:
#shuffle the list in place
random.shuffle(final_list)

In [10]:
len(final_list)

360

In [11]:
def make_yaml_file(combinations, idx_meta='0_20'):
    print(f'''
    {len(combinations)} pages will be created for subjective evalutation
    ''')
    
    final_dict_template = {
    'testname': 'AB Test',
     'testId': 'ab_noloop',
     'bufferSize': 2048,
     'stopOnErrors': True,
     'showButtonPreviousPage': True,
     'remoteService': 'service/write.php',
     'pages': []
    }
    #final page which sends results
    final_page_template =  {'type': 'finish',
       'name': 'Thank you',
       'content': 'Thank you for attending',
       'showResults': False,
       'writeResults': True,
       'questionnaire':[
           {
               'type':'text',
               'label': 'Name',
               'name': 'name'
           },
           {
               'type':'text',
               'label': 'Email',
               'name': 'email'
           },
           {
               'type':'likert',
               'name':'age',
               'label':'Age',
               'response':[
                   {'value': '0-20',
                   'label':'0 to 20',},
                    {'value': '20-40',
                   'label':'20 to 40',},
                    {'value': '20-40',
                   'label':'40 to 60',},
                    {'value': '40-60',
                   'label':'40 to 60',}
                   
               ]
           },
           {
               'type':'likert',
               'name':'gender',
               'label':'Gender',
               'response':[
                   {'value': 'female',
                   'label':'Female',},
                    {'value': 'male',
                   'label':'Male',},
                    {'value': 'other',
                   'label':'Other',},
               ]
           },
               {
               'type':'likert',
               'name':'subjective_eval_ever',
               'label':'Have you participated in audio subjective evailation before',
               'response':[
                   {'value': 'yes',
                   'label':'Yes',},
                    {'value': 'no',
                   'label':'No',},
               ]
           }
       ]
    }
    
    
    welcome_page_template = {
        'type': 'generic',
        'id': 'first_page',
        'name': 'Welcome',
        'content': '<b>Welcome to AB testing framework for DRONE ego noise ehancement. You will be presented a reference audio clip and denoised audio clip from two separate algorithms. You will be required to select one of the two clips which sounds the closest to the reference audio clip. </b>'
    }
    pages=[]
    pages.append(welcome_page_template)
    for i in range(1, len(combinations)):
        pages_dict_template = {'type': 'paired_comparison',
         'id': 'trialAB2',
         'name': None,
         'unforced': None,
         'content': 'test description AB',
         'showWaveform': True,
         'enableLooping': False,
         'reference': '',
         'stimuli': {'C1': '',
          'C2': ''}}
        pages.append(pages_dict_template)
        pages[i]['reference'] = combinations[i][0]
        pages[i]['stimuli']['C1'] = combinations[i][1]
        pages[i]['stimuli']['C2'] = combinations[i][2]
    
    pages.append(final_page_template)
    final_dict_template['pages'] = pages
    
    # dict to yaml
    yam_file_path = f'/Users/ashishalex/Documents/research/web_drone_project/pymushra/pymushra/static/yamls/other_model_combinations/idx_{idx_meta}.yaml'
    with open(yam_file_path, 'w') as f:
        yaml.dump(final_dict_template, f)
    print('Created yaml file at: ', yam_file_path)
    
    return final_dict_template

In [12]:
#window paramerter defines how many combinations (Reference, A, B) will the participant see
window = 20
start  = 0
ranges = []
while start < len(final_list):
    if start + window > len(final_list):
        ranges.append([start, len(final_list)])
        start+=window
    else:
        ranges.append([start, start+ window])
        start +=window
    

In [13]:
for start, end in ranges:
    out = make_yaml_file(combinations=final_list[start:end], idx_meta=f'{start}_{end}')


    20 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/pymushra/pymushra/static/yamls/other_model_combinations/idx_0_20.yaml

    20 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/pymushra/pymushra/static/yamls/other_model_combinations/idx_20_40.yaml

    20 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/pymushra/pymushra/static/yamls/other_model_combinations/idx_40_60.yaml

    20 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/pymushra/pymushra/static/yamls/other_model_combinations/idx_60_80.yaml

    20 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/pymushra/pymushra

In [14]:
len(final_list)

360